<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [4]:
import folium
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [7]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [11]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Coordinates 
CCAFS_LC40_coord = [28.562302, -80.577356]
CCAFS_SLC40_coord = [28.563197,	-80.576820]
KSC_KC39A_coord = [28.573255,	-80.646895]
VAFB_SLC4E_coord = [34.632834, -120.610745]

# List of coordinates and labels
launch_sites = [
    (CCAFS_LC40_coord, 'Cape Canaveral Air Force Station Launch Complex 40', 'CCAFS LC-40'),
    (CCAFS_SLC40_coord, 'Cape Canaveral Air Force Station Space Launch Complex 40', 'CCAFS SLC-40'),
    (KSC_KC39A_coord, 'Kennedy Space Center Launch Complex 39A', 'KSC LC-39A'),
    (VAFB_SLC4E_coord, 'Vandenberg Air Force Base Space Launch Complex 4E', 'VAFB SLC-4E')
]

# Add circles and markers to the map
for coord, popup_text, label in launch_sites:
    circle = folium.Circle(coord, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(popup_text))
    marker = folium.Marker(coord, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html=f'<div style="font-size: 12; color:#d35400;"><b>{label}</b></div>'))
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

In [ ]:
# Hint given to get started
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [12]:
# Task 2: Mark the success/failed launches for each site on the map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:
# Hint given to get started
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [16]:
# Define the function to determine marker color
def get_marker_color(class_value):
    return 'green' if class_value == 1 else 'red'

# Apply the function to create the new column
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# Display the first few rows to verify the changes
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Hint given to get started
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [17]:
# Iterate over each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinates and customize the icon color
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  
        popup=record['Launch Site'],  
        icon=folium.Icon(color=record['marker_color'])
    )
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [18]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

# Define the formatter function for the coordinates
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Create an instance of MousePosition with the desired parameters
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add the MousePosition to the map
site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [46]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Compute differences in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

     # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance



*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# Instructions and hint given:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [27]:
# Coordinates of closest coastline to CCAFS SLC-40:
# Lat: 28.56281 Lon: -80.56788
launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.56281
coastline_lon = -80.56788
distance_coastline_to_CCAFS_SLC40 = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Print the distance
print(f"The distance from CCAFS SLC-40 to the closest coastline is {distance_coastline_to_CCAFS_SLC40:.2f} km")

The distance from CCAFS SLC-40 to the closest coastline is 0.87 km


In [ ]:
## Hint/instructions to get started
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [29]:
# Add a marker for the closest coastline point
coastlinemarker1 = folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup=f"Closest Coastline\nDistance: {distance_coastline_to_CCAFS_SLC40:.2f} km",
    icon=folium.Icon(color='blue', icon='info-sign')
)

# Add the marker to the map
site_map.add_child(coastlinemarker1)

# Display the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
## Hint/instructions given 
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


In [31]:
# Create a 'folium.PolyLine' between the launch site and the coastline coordinates
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines1 = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines1)

# Display the map
site_map

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
#Instructions/hint given
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [32]:
# CCAFS SLC-40 coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820

#Closest city coordinates (Cape Canaveral)
city_lat = 28.46
city_lon = -80.53

#Closest railway coordinates
railway_lat = 28.56391
railway_lon = -80.5868

#Closest highway coordinates
highway_lat = 28.56295
highway_lon = -80.57075

distance_city_to_CCAFS_SLC40 = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_railway_to_CCAFS_SLC40 = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway_to_CCAFS_SLC40 = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)


# Print the distance
print(f"The distance from CCAFS SLC-40 to the closest city is {distance_city_to_CCAFS_SLC40:.2f} km")
print(f"The distance from CCAFS SLC-40 to the closest railway is {distance_railway_to_CCAFS_SLC40:.2f} km")
print(f"The distance from CCAFS SLC-40 to the closest highway is {distance_highway_to_CCAFS_SLC40:.2f} km")

The distance from CCAFS SLC-40 to the closest city is 12.36 km
The distance from CCAFS SLC-40 to the closest railway is 0.98 km
The distance from CCAFS SLC-40 to the closest highway is 0.59 km


In [33]:
# Used Copilot's help to create a dictionary of locations/distances, and for loops to add the markers and lines for each location
# Dictionary of locations with their coordinates and distances
locations = {
    'City': {'coords': [28.46, -80.53], 'distance': calculate_distance(28.563197, -80.576820, 28.46, -80.53), 'color': 'green'},
    'Railway': {'coords': [28.56391, -80.58683], 'distance': calculate_distance(28.563197, -80.576820, 28.56391, -80.58683), 'color': 'red'},
    'Highway': {'coords': [28.56295, -80.57075], 'distance': calculate_distance(28.563197, -80.576820, 28.56295, -80.57075), 'color': 'yellow'}
}

# Add markers and lines for each location
for name, info in locations.items():
    # Add marker
    marker = folium.Marker(
        location=info['coords'],
        popup=f"Closest {name}\nDistance: {info['distance']:.2f} km",
        icon=folium.Icon(color=info['color'], icon='info-sign')
    )
    site_map.add_child(marker)

 # Add line
    line = folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], info['coords']],
        weight=2,
        color=info['color']
    )
    site_map.add_child(line)

# Display the map
site_map

<ipython-input-33-a591ea7c71d7>:14: UserWarning: color argument of Icon should be one of: {'lightred', 'green', 'darkblue', 'lightblue', 'gray', 'orange', 'darkpurple', 'purple', 'beige', 'black', 'blue', 'lightgray', 'red', 'cadetblue', 'darkred', 'pink', 'darkgreen', 'white', 'lightgreen'}.
  icon=folium.Icon(color=info['color'], icon='info-sign')


In [40]:
#Now let's look at the distances for VAFB
# VAFB SLC-4E coordinates
launch_site_lat2 = 34.632834
launch_site_lon2 = -120.610745

#Closest city coordinates (Lompoc)
city_lat2 = 34.63667
city_lon2 = -120.45729

#Closest railway coordinates
railway_lat2 = 34.63431
railway_lon2 = -120.62444

#Closest highway coordinates
highway_lat2 = 34.63582
highway_lon2 = -120.43255

#Closest coast coordinates 
coast_lat2 = 34.6348
coast_lon2 = -120.62534

distance_city_to_VAFB = calculate_distance(launch_site_lat2, launch_site_lon2, city_lat2, city_lon2)
distance_railway_to_VAFB = calculate_distance(launch_site_lat2, launch_site_lon2, railway_lat2, railway_lon2)
distance_highway_to_VAFB = calculate_distance(launch_site_lat2, launch_site_lon2, highway_lat2, highway_lon2)
distance_coast_to_VAFB = calculate_distance(launch_site_lat2, launch_site_lon2, coast_lat2, coast_lon2)

# Print the distance
print(f"The distance from VAFB to the closest city is {distance_city_to_VAFB:.2f} km")
print(f"The distance from VAFB to the closest railway is {distance_railway_to_VAFB:.2f} km")
print(f"The distance from VAFB to the closest highway is {distance_highway_to_VAFB:.2f} km")
print(f"The distance from VAFB to the closest coast is {distance_coast_to_VAFB:.2f} km")

The distance from VAFB to the closest city is 14.05 km
The distance from VAFB to the closest railway is 1.26 km
The distance from VAFB to the closest highway is 16.31 km
The distance from VAFB to the closest coast is 1.35 km


In [47]:
# Now let's write this more consisely using a loop
# VAFB SLC-4E coordinates
launch_site_lat2 = 34.632834
launch_site_lon2 = -120.610745

# Dictionary of closest locations with their coordinates
locations = {
    "city": (34.63667, -120.45729),
    "railway": (34.63431, -120.62444),
    "highway": (34.63582, -120.43255),
    "coast": (34.6348, -120.62534)
}

# Iterate through the locations and calculate distances
for location, coords in locations.items():
    distance = calculate_distance(launch_site_lat2, launch_site_lon2, coords[0], coords[1])
    print(f"The distance from VAFB to the closest {location} is {distance:.2f} km")


The distance from VAFB to the closest city is 14.05 km
The distance from VAFB to the closest railway is 1.26 km
The distance from VAFB to the closest highway is 16.31 km
The distance from VAFB to the closest coast is 1.35 km


In [49]:
# Iterate through the locations, add markers, and draw lines
# List of colors to use for the lines
colors = ['blue', 'green', 'purple', 'orange']

for i, (location, coords) in enumerate(locations.items()):
    # Calculate the distance
    distance = calculate_distance(launch_site_lat2, launch_site_lon2, coords[0], coords[1])
    
    # Add a marker for each location with distance in the popup
    folium.Marker(
        location=coords,
        popup=f'Closest {location} ({distance:.2f} km)',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)
    
    # Draw a line between the launch site and each location with different colors
    folium.PolyLine(
        locations=[(launch_site_lat2, launch_site_lon2), coords],
        color=colors[i % len(colors)],
        weight=2.5,
        opacity=1
    ).add_to(site_map)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
